#  📚 3.6相恋篇-Keras自定义类


         ✅能今天做好的事就不要等到明天。以梦为马，学习趁年华.

----------------------------------------------------------------------------------------------------------------

        尽管 Keras 提供了很多的常用网络层，但深度学习可以使用的网络层远远不止这些经 典的网络层，对于需要创建自定义逻辑的网络层，可以通过自定义类来实现。在创建自定 义网络层类时，需要继承自 layers.Layer 基类;创建自定义的网络类，需要继承自keras.Model 基类，这样产生的自定义类才能够方便的利用 Layer/Model 基类提供的参数管理功能，同时也能够与其他的标准网络层类交互使用。

# 一、本节目标
        本节将讲述自定义层和自定义网络。会自己自定义层和网络。

# 二、自定义网络层
  

    对于自定义的网络层，需要实现初始化__init__方法和前向传播逻辑 call 方法。我们以某个具体的自定义网络层为例，假设我们需要一个没有偏置的全连接层，即 bias 为 0，同 时固定激活函数为 ReLU 函数。尽管这可以通过标准的 Dense 层创建，但我们还是实现这个自定义类。
首先创建类并继承自 Layer 基类，创建初始化方法，并调用母类的初始化函数，由于 是全连接层，我们需要设置特征的长度 inp_dim 和输出特征的长度 outp_dim，并通过self.add_variable(name, shape)创建 shape 大小，名字为 name 的张量，并设置为需要优化:

In [2]:
from tensorflow.keras import layers
class MyDense(layers.Layer): # 自定义网络层
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()
    # 创建权值张量并添加到类管理列表中，设置为需要优化
        self.kernel = self.add_variable('w', [inp_dim, outp_dim],trainable=True)
net = MyDense(4,3) # 创建输入为 4，输出为 3 节点的自定义层
net.variables

[<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.10120511,  0.7540673 , -0.19999331],
        [-0.5073224 ,  0.80730104, -0.3325972 ],
        [ 0.6530596 ,  0.8402846 , -0.8346729 ],
        [-0.2951159 , -0.06259227,  0.6104456 ]], dtype=float32)>]

In [3]:
net.trainable_variables

[<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.10120511,  0.7540673 , -0.19999331],
        [-0.5073224 ,  0.80730104, -0.3325972 ],
        [ 0.6530596 ,  0.8402846 , -0.8346729 ],
        [-0.2951159 , -0.06259227,  0.6104456 ]], dtype=float32)>]

# 三、自定义网络

    在完成了我们自定义的全连接层类之后，我们基于上述的“无偏置的全连接层”来实现 MNIST 手写数字图片模型的创建。自定义的类可以和其他标准类一样，通过 Sequential 容器方便地包裹成一个网络模型:

In [7]:
import tensorflow as tf
network =tf.keras.models.Sequential([MyDense(784, 256), # 使用自定义的层 MyDense(256, 128),
                 MyDense(128, 64),
                 MyDense(64, 32),
                 MyDense(32, 10)])
network.build(input_shape=(None, 28*28)) 


In [8]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_6 (MyDense)         (None, 784)               200704    
_________________________________________________________________
my_dense_7 (MyDense)         (None, 784)               8192      
_________________________________________________________________
my_dense_8 (MyDense)         (None, 784)               2048      
_________________________________________________________________
my_dense_9 (MyDense)         (None, 784)               320       
Total params: 211,264
Trainable params: 211,264
Non-trainable params: 0
_________________________________________________________________


     可以看到，通过堆叠我们的自定义类，一样可以实现 5 层的全连接层网络，每层全连接层 无偏置张量，同时激活函数固定使用 ReLU。更普遍地，我们可以继承基类来实现任意逻辑的自定义网络类。下面我们来创建自定 义网络类，首先创建并继承 Model 基类，分布创建对应的网络层对象:

In [9]:
from tensorflow import keras
class MyModel(keras.Model):
    # 自定义网络类，继承自 Model 基类 
    def __init__(self):
        super(MyModel, self).__init__() # 完成网络内需要的网络层的创建工作 self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64) 
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
        #然后实现自定义网络的前向运算逻辑:
    def call(self, inputs, training=None): # 自定义前向运算逻辑
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x) 
        x = self.fc5(x)
        return x

这个例子可以直接使用第一种方式通过 Sequential 容器包裹。但是由于 Sequential 在前向传播是依次调用每个网络层的前向传播函数，灵活性一般，而自定义网络的前向逻辑可以任意定制，两者各有优缺点。

#  四、课后作业

1 会自己写网络层，可以试着自己用tensorflow2.0写一写常用的模型。